In [1]:
# Imports
import torch.nn as nn

In [3]:
# Task 0. Generator
class Generator(nn.Module):
    """
    Generator subclass for the generator side of the network
    """
    def __init__(self,
                 input_size,
                 hidden_size,
                 output_size):
        """
        Initialize Generator class

        Args:
            input_size - integer size of the input tensor
            hidden_size - integer size of the hidden layer
            output_size - integer size of the output layer
        """
        super(Generator, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, output_size),
        )


        def forward(self, x):
            """
            Forward pass of the Generator network

            Args:
                x - Input tensor of shape (batch_size, input_size)

            Returns:
                Output tensor of shape (batch_size, output_size)
            """
            return self.layers(x)


In [4]:
# 0-main
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
genTester = __import__('0-generator').Generator

G_Test = genTester(1,1,1)

print(G_Test)

Generator(
  (layers): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
    (1): Tanh()
    (2): Linear(in_features=1, out_features=1, bias=True)
    (3): Tanh()
    (4): Linear(in_features=1, out_features=1, bias=True)
  )
)
